In [2]:
%defaultDatasource jdbc:h2:mem:db

# Criação da Tabela de Players

In [3]:
DROP Table IF EXISTS Players;

CREATE TABLE Players (
    Username VARCHAR(25) NOT NULL,
    Title VARCHAR(3),
    Bullet INTEGER,
    Blitz INTEGER,
    Rapid INTEGER,
    SiteName VARCHAR(10) NOT NULL,
    PRIMARY KEY(Username, SiteName)
) AS SELECT
    username,
    title,
    bullet_rating,
    blitz_rating,
    rapid_rating,
    site_name
    FROM CSVREAD('../data/processed/all_player_list.csv')

In [3]:
SELECT * FROM Players

# Criação da Tabela de Jogos

In [4]:
DROP Table IF EXISTS Games;

CREATE TABLE Games (
    Id VARCHAR(40) NOT NULL,
    Ranked BOOLEAN NOT NULL,
    Time_control VARCHAR(20) NOT NULL,
    Result VARCHAR(20) NOT NULL,
    Winner VARCHAR(25),
    White VARCHAR(25) NOT NULL,
    Black VARCHAR(25) NOT NULL,
    Moves VARCHAR(9999),
    Openings VARCHAR(95),
    SiteName VARCHAR(10) NOT NULL,
    PRIMARY KEY(Id),
) AS SELECT
    id,
    ranked,
    time_control,
    status,
    winner,
    white,
    black,
    moves,
    opening,
    site_name
    FROM CSVREAD('../data/processed/all_games.csv')

In [7]:
SELECT * FROM Games

# Criação da Tabela de Openings

In [5]:
DROP Table IF EXISTS Openings;

CREATE TABLE Openings (
    Eco VARCHAR(3),
    Opening VARCHAR(95),
    Pgn VARCHAR(210),
    PRIMARY KEY (Pgn)
) AS SELECT
    eco,
    name,
    pgn
    FROM CSVREAD('../data/processed/openings.csv')

In [9]:
SELECT * FROM Openings

# Análises realizadas no Database

In [6]:
SELECT * FROM Games G
    LEFT JOIN Players PW
    ON G.white = PW.Username AND G.SiteName = PW.SiteName
    LEFT JOIN Players PB
    ON G.black = PB.Username AND G.SiteName = PB.SiteName
    

In [6]:
DROP TABLE IF EXISTS Winner;

CREATE VIEW Winner AS
SELECT P.Username, COUNT(*) Ganhadores
    FROM Games G, Players P
    WHERE G.Winner = P.Username AND G.SiteName = 'lichess'
    GROUP BY P.Username;

SELECT P.Username, P.Title, P.SiteName, W.Ganhadores
FROM Winner W, Players P
WHERE W.Username = P.Username AND W.Ganhadores = (SELECT MAX(Winner.Ganhadores) FROM Winner)

In [7]:
DROP TABLE IF EXISTS MaxOpenings;

CREATE VIEW MaxOpenings AS
SELECT G.Openings, COUNT(*) Aberturas
    FROM Games G, Players P
    WHERE P.Username = 'kirillgenius' AND (P.Username = G.White OR P.Username = G.Black)
    GROUP BY G.Openings
    ORDER BY COUNT(*) DESC;

SELECT * FROM MaxOpenings

In [8]:
SELECT G.result, COUNT (*) OCORRENCIAS
FROM Games G
GROUP BY G.result
ORDER BY OCORRENCIAS DESC

In [12]:
SELECT X.winner, COUNT (*) WINS
FROM (SELECT G.result, G.winner,G.white,G.black
FROM Games G
WHERE G.winner = G.white OR G.winner = G.Black) X
GROUP BY X.winner
ORDER BY WINS DESC;